In [1]:
!pip install requests beautifulsoup4 pandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import random

In [5]:
def scrape_topjobs():
    """
    Scrape job listings from TopJobs.lk and save to Excel file
    """
    url = "https://www.topjobs.lk/applicant/vacancybyfunctionalarea.jsp?FA=IT"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    }
    
    print("Starting to scrape TopJobs.lk...")
    
    try:
        time.sleep(random.uniform(1, 3))
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        print("Successfully fetched the webpage")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Let's debug what we're getting
    print("=== DEBUGGING INFO ===")
    
    # Check for different types of links
    all_links = soup.find_all('a')
    vacancy_links = [link for link in all_links if link.get('href') and ('vacancy' in link.get('href', '') or 'job' in link.get('href', ''))]
    print(f"Found {len(all_links)} total links")
    print(f"Found {len(vacancy_links)} potential job links")
    
    # Check the first few vacancy links
    if vacancy_links:
        print("\nFirst 3 job links found:")
        for i, link in enumerate(vacancy_links[:3]):
            print(f"{i+1}. Text: '{link.text.strip()}' | Href: '{link.get('href')}'")
            # Check parent elements
            parent = link.parent
            if parent:
                print(f"   Parent tag: {parent.name}")
                if parent.name == 'td':
                    row = parent.parent
                    if row and row.name == 'tr':
                        print(f"   Row content: {row.get_text(strip=True)[:100]}...")
    
    # Try different approaches to find job data
    job_list = []
    
    # Method 1: Look for all links with 'vacancy' in href
    print("\n=== METHOD 1: Processing vacancy links ===")
    for link in vacancy_links:
        try:
            job_title = link.text.strip()
            if not job_title or len(job_title) < 3:
                continue
                
            # Get the table row containing this link
            current_element = link
            row = None
            
            # Traverse up to find the table row
            while current_element and current_element.name != 'tr':
                current_element = current_element.parent
                if current_element is None:
                    break
            
            row = current_element
            
            if row:
                # Look for company name in the same row
                company_element = None
                
                # Try different approaches to find company name
                fonts = row.find_all('font')
                for font in fonts:
                    if font.get('class') == ['mini'] or 'mini' in str(font.get('class', [])):
                        company_element = font
                        break
                
                # If no company found with 'mini' class, try other fonts
                if not company_element and fonts:
                    # Look for font elements that are not the job title
                    for font in fonts:
                        font_text = font.get_text(strip=True)
                        if font_text and font_text != job_title and len(font_text) > 2:
                            company_element = font
                            break
                
                # Try looking for company in table cells
                if not company_element:
                    cells = row.find_all('td')
                    for cell in cells:
                        cell_text = cell.get_text(strip=True)
                        if cell_text and cell_text != job_title and len(cell_text) > 2 and not cell_text.startswith('http'):
                            company_element = cell
                            break
                
                if company_element:
                    company_name = company_element.get_text(strip=True)
                    
                    # Build job link
                    job_link = link.get('href', '')
                    if job_link and not job_link.startswith('http'):
                        job_link = f"https://www.topjobs.lk{job_link}"
                    
                    # Look for location info
                    row_text = row.get_text()
                    location = "Not specified"
                    sri_lankan_cities = ["Colombo", "Kandy", "Galle", "Negombo", "Matara", "Kurunegala", "Ratnapura", "Badulla", "Anuradhapura", "Trincomalee"]
                    
                    for city in sri_lankan_cities:
                        if city in row_text:
                            location = city
                            break
                    
                    job_data = {
                        "Job Title": job_title,
                        "Company": company_name,
                        "Location": location,
                        "Job Link": job_link,
                        "Scraped Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    }
                    
                    job_list.append(job_data)
                    
                    # Show first few jobs being processed
                    if len(job_list) <= 3:
                        print(f"Job {len(job_list)}: {job_title} at {company_name}")
                        
        except Exception as e:
            print(f"Error processing job: {e}")
            continue
    
    print(f"\nTotal jobs extracted: {len(job_list)}")
    
    if not job_list:
        print("\n=== DEBUGGING: Let's examine the page structure ===")
        
        # Show some sample table rows
        tables = soup.find_all('table')
        print(f"Found {len(tables)} tables")
        
        for i, table in enumerate(tables[:3]):
            rows = table.find_all('tr')
            print(f"\nTable {i+1} has {len(rows)} rows")
            if rows:
                for j, row in enumerate(rows[:3]):
                    print(f"  Row {j+1}: {row.get_text(strip=True)[:150]}...")
        
        return None
    
    # Create DataFrame
    df = pd.DataFrame(job_list)
    
    # Remove duplicates based on job title and company
    df = df.drop_duplicates(subset=['Job Title', 'Company'], keep='first')
    print(f"After removing duplicates: {len(df)} jobs")
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    excel_filename = f"topjobs_listings_{timestamp}.xlsx"
    
    try:
        # Save to Excel
        with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Job Listings', index=False)
            
            workbook = writer.book
            worksheet = writer.sheets['Job Listings']
            
            # Auto-adjust column widths
            for column in worksheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = min(max_length + 2, 50)
                worksheet.column_dimensions[column_letter].width = adjusted_width
        
        print(f"Job data saved to Excel file: {excel_filename}")
        
    except Exception as e:
        print(f"Error saving to Excel: {e}")
        df.to_csv(f"topjobs_listings_{timestamp}.csv", index=False)
        print(f"Job data saved to CSV file instead")
    
    # Display summary
    print(f"\n--- Scraping Summary ---")
    print(f"Total jobs found: {len(df)}")
    print(f"Unique companies: {df['Company'].nunique()}")
    
    return df

# Run the scraper
df = scrape_topjobs()

# Display the results in Jupyter
if df is not None:
    print("\n--- Sample Jobs ---")
    display(df.head(10))  # Show first 10 jobs
    
    # You can also create visualizations
    print(f"\nTop 10 Companies by Job Count:")
    company_counts = df['Company'].value_counts().head(10)
    display(company_counts)
else:
    print("No jobs were scraped. Check the debugging output above.")

Starting to scrape TopJobs.lk...
Successfully fetched the webpage
=== DEBUGGING INFO ===
Found 68 total links
Found 47 potential job links

First 3 job links found:
1. Text: 'IT-Sware/DB/QA/Web/Graphics/GIS' | Href: 'vacancybyfunctionalarea.jsp?FA=SDQ&jst=OPEN'
   Parent tag: td
   Row content: IT-Sware/DB/QA/Web/Graphics/GISIT-HWare/Networks/SystemsAccounting/Auditing/FinanceBanking & Finance...
2. Text: 'IT-HWare/Networks/Systems' | Href: 'vacancybyfunctionalarea.jsp?FA=HNS&jst=OPEN'
   Parent tag: td
   Row content: IT-Sware/DB/QA/Web/Graphics/GISIT-HWare/Networks/SystemsAccounting/Auditing/FinanceBanking & Finance...
3. Text: 'Accounting/Auditing/Finance' | Href: 'vacancybyfunctionalarea.jsp?FA=ACA&jst=OPEN'
   Parent tag: td
   Row content: IT-Sware/DB/QA/Web/Graphics/GISIT-HWare/Networks/SystemsAccounting/Auditing/FinanceBanking & Finance...

=== METHOD 1: Processing vacancy links ===
Job 1: IT-Sware/DB/QA/Web/Graphics/GIS at IT-HWare/Networks/Systems
Job 2: IT-HWare/Networks/Sys

,Job Title,Company,Location,Job Link,Scraped Date
0,IT-Sware/DB/QA/Web/Graphics/GIS,IT-HWare/Networks/Systems,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
1,IT-HWare/Networks/Systems,IT-Sware/DB/QA/Web/Graphics/GIS,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
2,Accounting/Auditing/Finance,IT-Sware/DB/QA/Web/Graphics/GIS,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
3,Banking & Finance/Insurance,IT-Sware/DB/QA/Web/Graphics/GIS,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
4,Sales/Marketing/Merchandising,IT-Sware/DB/QA/Web/Graphics/GIS,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
5,HR/Training,Corporate Management/Analysts,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
6,Corporate Management/Analysts,HR/Training,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
7,Office Admin/Secretary/Receptionist,HR/Training,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
8,Civil Eng/Interior Design/Architecture,HR/Training,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56
9,IT-Telecoms,HR/Training,Not specified,https://www.topjobs.lkvacancybyfunctionalarea....,2025-06-06 21:02:56



Top 10 Companies by Job Count:


Company
IT-Sware/DB/QA/Web/Graphics/GIS        4
HR/Training                            4
Customer Relations/Public Relations    4
Agriculture/Dairy/Environment          4
Supervision/Quality Control            4
Hotel/Restaurant/Hospitality           4
Logistics/Warehouse/Transport          1
Corporate Management/Analysts          1
IT-HWare/Networks/Systems              1
Travel/Tourism                         1
Name: count, dtype: int64